In [1]:
import config

In [2]:
import spotipy
import json
import pprint
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
from pyjarowinkler import distance
from spotipy.oauth2 import SpotifyClientCredentials


#Initialize SpotiPy with user credentias
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id= config.c_id,
                                                           client_secret= config.c_se))

### Getting playlists

In [3]:
# locating playlist
playlist_country = sp.user_playlist_tracks("spotify", "3RPaWvjgg1SAi8oIh09Irp",market="GB")
playlist_rock = sp.user_playlist_tracks("spotify", "7KMtxb9kh3TtVvJTdLtJ2K",market="GB")
playlist_house = sp.user_playlist_tracks("spotify", "4VXFwpNvQysB8yS6E2t2ls",market="GB")
playlist_classicals = sp.user_playlist_tracks("spotify", "1h0CEZCm6IbFTbxThn6Xcs",market="GB")
playlist_dance = sp.user_playlist_tracks("spotify", "794tEP1QrKMcaQcFmEyFst",market="GB")
playlist_hiphop = sp.user_playlist_tracks("spotify", "0agbFCQZvK4q9WR0d4Iqcj",market="GB")
playlist_pop = sp.user_playlist_tracks("spotify", "2YRe7HRKNRvXdJBp9nXFza",market="GB")
playlist_jazz = sp.user_playlist_tracks("spotify", "7g2KyiE9RI5F1fV60OIUu4",market="GB")

### Getting tracks

In [4]:
track=[]

In [5]:
# extracting tracks from playlist
def get_playlist_tracks(username, playlist_id):
    results = sp.user_playlist_tracks(username,playlist_id,market="GB")
    tracks = results['items']
    while results['next']:
        results = sp.next(results)
        tracks.extend(results['items'])
    return tracks

### Getting artists

In [6]:
artist=[]

In [7]:
# extracting artists from track
def get_artists_from_track(track):
    return [artist["name"] for artist in track["artists"]]

In [8]:
# extracting artists from playlist
def get_artists_from_playlist(playlist_id):
    tracks_from_playlist = get_playlist_tracks("spotify", playlist_id)
    return list(get_artists_from_track(track["track"]) for track in tracks_from_playlist)

### Getting track ids

In [9]:
track_id=[]

### Combining data

In [10]:
track_df = pd.DataFrame({'title':track,
                     'artist':artist,
                     'id':track_id})

In [11]:
def append_info(playlist_id):
    global track_df
    new_track_list = get_playlist_tracks("spotify", playlist_id)
    new_artist_list = get_artists_from_playlist(playlist_id)
    new_track=[]
    new_artist=[]
    new_track_id=[]
    for i in range(0, len(new_track_list)):
        new_track.append(new_track_list[i]['track']['name'])
    for i in new_artist_list:
        new_artist.append(", ".join(i))
    for i in range(0, len(new_track_list)):
        new_track_id.append(new_track_list[i]['track']['id']) 
    new_df = pd.DataFrame({'title':new_track,
                         'artist':new_artist,
                         'id':new_track_id})
    track_df = track_df.append(new_df)    

In [12]:
append_info("3RPaWvjgg1SAi8oIh09Irp")

In [13]:
append_info("7KMtxb9kh3TtVvJTdLtJ2K")

In [14]:
append_info("4VXFwpNvQysB8yS6E2t2ls")

In [15]:
append_info("1h0CEZCm6IbFTbxThn6Xcs")

In [16]:
append_info("794tEP1QrKMcaQcFmEyFst")

In [17]:
append_info("0agbFCQZvK4q9WR0d4Iqcj")

In [18]:
append_info("2YRe7HRKNRvXdJBp9nXFza")

In [19]:
append_info("7g2KyiE9RI5F1fV60OIUu4")

In [20]:
track_df

title  \
0         Crazy - Single Version   
1            Folsom Prison Blues   
2         I Will Always Love You   
3                    Springsteen   
4    He Stopped Loving Her Today   
..                           ...   
182   Doozy - 1966 Album Version   
183                   Whispering   
184                    Pannonica   
185                  Free Spirit   
186             (It's Just) Talk   

                                                artist                      id  
0                         Patsy Cline, The Jordanaires  3zpj9dvJABiyMrmLCPw6i8  
1                       Johnny Cash, The Tennessee Two  121sjkub4p4zcFNozkKnQD  
2                                         Dolly Parton  6nRvgo4ySezQKD1D7NIqAu  
3                                          Eric Church  05ZACkzW8YbRq3eFgIfSNB  
4                                         George Jones  6qMgx8ys4MNNxypaIvbG0i  
..                                                 ...                     ...  
182                                       Benny Carter  0iZFlr1KifqE2h5jM8g5pF  
183                                        Miles Davis  4Bb7eBhfbcDMYMn8bGCyNh  
184  Thelonious Monk, Sonny Rollins, Ernie Henry, C...  3EM4Vr12ZTUee0OLjTw01F  
185                                    Ibrahim Maalouf  3qYeDicgcx84m036Nkmz7f  
186                                  Pat Metheny Group  1NIQIdAXQBcXBzd6wkpTqF  

[8969 rows x 3 columns]

In [22]:
track_df.title.duplicated

<bound method Series.duplicated of 0           Crazy - Single Version
1              Folsom Prison Blues
2           I Will Always Love You
3                      Springsteen
4      He Stopped Loving Her Today
                  ...             
182     Doozy - 1966 Album Version
183                     Whispering
184                      Pannonica
185                    Free Spirit
186               (It's Just) Talk
Name: title, Length: 8969, dtype: object>